In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

2023-02-09 17:10:31.545304: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 17:10:31.730934: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-09 17:10:32.527106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64:/usr/local/cuda-11.1/lib64:
2023-02-09 17:10:32.527246: W te

In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')

In [3]:
embed_file = 'embed.pickle'
import pickle5 as p
with open(embed_file, "rb") as fh:
    embed_df = p.load(fh)
embed_df.head()

,e_0,e_1,e_2,e_3,e_4,e_5,e_6,e_7,e_8,e_9,e_10,e_11,e_12,e_13,e_14,e_15,e_16,e_17,e_18,e_19,e_20,e_21,e_22,e_23,e_24,e_25,e_26,e_27,e_28,e_29,e_30,e_31,e_32,e_33,e_34,e_35,e_36,e_37,e_38,e_39,e_40,e_41,e_42,e_43,e_44,e_45,e_46,e_47,e_48,e_49,e_50,e_51,e_52,e_53,e_54,e_55,e_56,e_57,e_58,e_59,e_60,e_61,e_62,e_63,e_64,e_65,e_66,e_67,e_68,e_69,e_70,e_71,e_72,e_73,e_74,e_75,e_76,e_77,e_78,e_79,e_80,e_81,e_82,e_83,e_84,e_85,e_86,e_87,e_88,e_89,e_90,e_91,e_92,e_93,e_94,e_95,e_96,e_97,e_98,e_99,e_100,e_101,e_102,e_103,e_104,e_105,e_106,e_107,e_108,e_109,e_110,e_111,e_112,e_113,e_114,e_115,e_116,e_117,e_118,e_119,e_120,e_121,e_122,e_123,e_124,e_125,e_126,e_127,e_128,e_129,e_130,e_131,e_132,e_133,e_134,e_135,e_136,e_137,e_138,e_139,e_140,e_141,e_142,e_143,e_144,e_145,e_146,e_147,e_148,e_149,e_150,e_151,e_152,e_153,e_154,e_155,e_156,e_157,e_158,e_159,e_160,e_161,e_162,e_163,e_164,e_165,e_166,e_167,e_168,e_169,e_170,e_171,e_172,e_173,e_174,e_175,e_176,e_177,e_178,e_179,e_180,e_181,e_182,e_183,e_184,e_185,e_186,e_187,e_188,e_189,e_190,e_191,e_192,e_193,e_194,e_195,e_196,e_197,e_198,e_199,e_200,e_201,e_202,e_203,e_204,e_205,e_206,e_207,e_208,e_209,e_210,e_211,e_212,e_213,e_214,e_215,e_216,e_217,e_218,e_219,e_220,e_221,e_222,e_223,e_224,e_225,e_226,e_227,e_228,e_229,e_230,e_231,e_232,e_233,e_234,e_235,e_236,e_237,e_238,e_239,e_240,e_241,e_242,e_243,e_244,e_245,e_246,e_247,e_248,e_249,e_250,e_251,e_252,e_253,e_254,e_255,e_256,e_257,e_258,e_259,e_260,e_261,e_262,e_263,e_264,e_265,e_266,e_267,e_268,e_269,e_270,e_271,e_272,e_273,e_274,e_275,e_276,e_277,e_278,e_279,e_280,e_281,e_282,e_283,e_284,e_285,e_286,e_287,e_288,e_289,e_290,e_291,e_292,e_293,e_294,e_295,e_296,e_297,e_298,e_299,e_300,e_301,e_302,e_303,e_304,e_305,e_306,e_307,e_308,e_309,e_310,e_311,e_312,e_313,e_314,e_315,e_316,e_317,e_318,e_319,e_320,e_321,e_322,e_323,e_324,e_325,e_326,e_327,e_328,e_329,e_330,e_331,e_332,e_333,e_334,e_335,e_336,e_337,e_338,e_339,e_340,e_341,e_342,e_343,e_344,e_345,e_346,e_347,e_348,e_349,e_350,e_351,e_352,e_353,e_354,e_355,e_356,e_357,e_358,e_359,e_360,e_361,e_362,e_363,e_364,e_365,e_366,e_367,e_368,e_369,e_370,e_371,e_372,e_373,e_374,e_375,e_376,e_377,e_378,e_379,e_380,e_381,e_382,e_383,e_384,e_385,e_386,e_387,e_388,e_389,e_390,e_391,e_392,e_393,e_394,e_395,e_396,e_397,e_398,e_399,e_400,e_401,e_402,e_403,e_404,e_405,e_406,e_407,e_408,e_409,e_410,e_411,e_412,e_413,e_414,e_415,e_416,e_417,e_418,e_419,e_420,e_421,e_422,e_423,e_424,e_425,e_426,e_427,e_428,e_429,e_430,e_431,e_432,e_433,e_434,e_435,e_436,e_437,e_438,e_439,e_440,e_441,e_442,e_443,e_444,e_445,e_446,e_447,e_448,e_449,e_450,e_451,e_452,e_453,e_454,e_455,e_456,e_457,e_458,e_459,e_460,e_461,e_462,e_463,e_464,e_465,e_466,e_467,e_468,e_469,e_470,e_471,e_472,e_473,e_474,e_475,e_476,e_477,e_478,e_479,e_480,e_481,e_482,e_483,e_484,e_485,e_486,e_487,e_488,e_489,e_490,e_491,e_492,e_493,e_494,e_495,e_496,e_497,e_498,e_499,e_500,e_501,e_502,e_503,e_504,e_505,e_506,e_507,e_508,e_509,e_510,e_511,e_512,e_513,e_514,e_515,e_516,e_517,e_518,e_519,e_520,e_521,e_522,e_523,e_524,e_525,e_526,e_527,e_528,e_529,e_530,e_531,e_532,e_533,e_534,e_535,e_536,e_537,e_538,e_539,e_540,e_541,e_542,e_543,e_544,e_545,e_546,e_547,e_548,e_549,e_550,e_551,e_552,e_553,e_554,e_555,e_556,e_557,e_558,e_559,e_560,e_561,e_562,e_563,e_564,e_565,e_566,e_567,e_568,e_569,e_570,e_571,e_572,e_573,e_574,e_575,e_576,e_577,e_578,e_579,e_580,e_581,e_582,e_583,e_584,e_585,e_586,e_587,e_588,e_589,e_590,e_591,e_592,e_593,e_594,e_595,e_596,e_597,e_598,e_599,e_600,e_601,e_602,e_603,e_604,e_605,e_606,e_607,e_608,e_609,e_610,e_611,e_612,e_613,e_614,e_615,e_616,e_617,e_618,e_619,e_620,e_621,e_622,e_623,e_624,e_625,e_626,e_627,e_628,e_629,e_630,e_631,e_632,e_633,e_634,e_635,e_636,e_637,e_638,e_639,e_640,e_641,e_642,e_643,e_644,e_645,e_646,e_647,e_648,e_649,e_650,e_651,e_652,e_653,e_654,e_655,e_656,e_657,e_658,e_659,e_660,e_661,e_662,e_663,e_664,e_665,e_666,e_667,e_668,e_669,e_670,e_671,e_672,e_673,e_674,e_675,e_676,e_677,e_678,e_679,e_680,e_681,e_682,e_683,e_684

In [4]:
embed_df.shape

(407088, 768)

In [5]:
train_embed_df = embed_df[:len(train_df)]
test_embed_df = embed_df[len(train_df):]

In [6]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [7]:
train = np.concatenate([train_df[main_attrs].values, train_embed_df.values], axis=1)
test = np.concatenate([test_df[main_attrs].values, test_embed_df.values], axis=1)
print(train.shape)

(375734, 827)


In [8]:
import pickle
with open('train.pkl','wb') as f:
     pickle.dump(train, f)

In [9]:
with open('test.pkl','wb') as f:
     pickle.dump(test, f)

In [10]:
region_df = train_df.groupby(['lat', 'lon', 'climateregions__climateregion'])[target[0]].count().reset_index()
region_df['id'] = len(region_df)

In [11]:
train_df = train_df.merge(region_df[['lat', 'lon', 'climateregions__climateregion', 'id']], on=['lat', 'lon', 'climateregions__climateregion'], how='left')
test_df = test_df.merge(region_df[['lat', 'lon', 'climateregions__climateregion', 'id']], on=['lat', 'lon', 'climateregions__climateregion'], how='left')

In [13]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 60


In [15]:
train = pd.concat([train_df[main_attrs], train_embed_df], axis=1)
test = pd.concat([test_df[main_attrs], test_embed_df], axis=1)

(375734, 828)


In [19]:
features = [c for c in train.columns if c != 'climateregions__climateregion']

In [22]:
def model_wids2023():    
    inputs = []
    embeddings  = []
    for c in ['id']:
        num_unique_values = 514
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))               
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.5)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        embeddings.append(out)

    input_numeric1     = layers.Input(shape=(len(features),))
    embedding_numeric1 = layers.Dense(250, activation='relu')(input_numeric1)
    inputs.append(input_numeric1)
    embeddings.append(embedding_numeric1)
    
    
    x = layers.Concatenate()(embeddings)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128)(x)

    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)
    
    y = layers.Dense(1)(x)
    model = Model(inputs=inputs, outputs=y)
    return model

In [24]:

model = model_wids2023()

NameError: name 'layers' is not defined